<a href="https://colab.research.google.com/github/Talendar/br_fake_news_detection/blob/main/br_fake_news_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import os
import re

%tensorflow_version 2.x
import tensorflow as tf

from tensorflow.keras.callbacks import Callback
from IPython.display import clear_output

!git clone https://github.com/roneysco/Fake.br-Corpus
DATA_PATH = "./Fake.br-Corpus/size_normalized_texts"

In [153]:
class ClearCallback(Callback):
    """ Handles the cleaning of the log during the training of a model. """

    def __init__(self, current_k, total_k):
        self._current_k = current_k
        self._total_k = total_k

    def on_epoch_end(self, epoch, logs=None):
        """ Clears the log. Called when a training epoch ends. """
        clear_output(wait=True)
        print("Running %d-folds cross-validation. Current fold: %d.\n" % (self._total_k, self._current_k))

#### Loading data ####

In [93]:
def load_txts(path):
    txts = []
    for filename in sorted(os.listdir(path), key=lambda x: int(re.match("[0-9]+", x).group())):
        with open(os.path.join(path, filename)) as f:
            txts.append(f.read())
    return txts


true_txts = load_txts(os.path.join(DATA_PATH, "true"))
fake_txts = load_txts(os.path.join(DATA_PATH, "fake"))
assert(len(true_txts) == len(fake_txts))

data = pd.DataFrame( [{"text": t, "label": 0} for t in true_txts] + [{"text": f, "label": 1} for f in fake_txts] ).sample(frac=1)
pd.set_option('max_colwidth', 200)
data

,text,label
5009,"Após denúncia da Veja, irmã de Aécio publica vídeo e chora: ""Vamos provar que é mentira"". . Após ser denunciado por uma matéria na Revista Veja neste fim de semana, o senador Aécio Neves se mostr...",1
1664,Quatro pessoas ficam feridas em acidente entre carro e caminhão na BR-376. Motorista do automóvel sofreu ferimentos graves e foi resgatado pelo helicóptero do Samu nesta terça-feira (23). Em Saran...,0
2404,"Moro aceita denúncia contra Lula e outros 12 por caso envolvendo sítio em Atibaia. Segundo a força-tarefa da Lava Jato, as empreiteiras Odebrecht e OAS compraram e pagaram por melhorias no sítio c...",0
6795,Reunião urgente do Foro de SP ? Dilma chega à capital paulista para encontrar Lula. A presidente futura ex-presidente Dilma Rousseff nessa tarde desta em São Paulo para se encontrar com Lula. Di...,1
869,"No primeiro dia de 2017, um antigo e muito próximo espírito santo de orelha de Michel Miguel Elias Temer Lulia deu-lhe um conselho que ao chefe, então, pareceu promissor. Como não era de sua alçad...",0
...,...,...
2188,"Um corredor com segurança armado divide o gabinete do juiz Sérgio Moro da sala de audiências no segundo andar do edifício da Justiça Federal, em Curitiba. Uma câmera acoplada no computador registr...",0
874,"Os relatores de propostas que podem alterar o sistema eleitoral brasileiro nas próximas eleições admitiram o óbvio ontem, durante o “Fórum Estadão – Reforma Política em Debate”, realizado em parce...",0
3971,"Evandro Mesquita protesta e chama senadores de vermes: ""Querem calar Sérgio Moro!"". . Ao ver os problemas políticos e econômicos do Brasil, Mesquita mostrou que é um artista politizado e se manté...",1
3248,"Investigação mostra reais causas de alto preço do pedágio no Paraná, diz procurador da Lava Jato. 48ª fase foi deflagrada nesta quinta-feira (22); seis pessoas foram presas. . O procurador do Mini...",0


# BAG-OF-WORDS

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('rslp')

import string
from sklearn.feature_extraction.text import CountVectorizer

STOPWORDS = nltk.corpus.stopwords.words('portuguese')
STEMMER = nltk.stem.RSLPStemmer()

In [132]:
def normalize_texts(corpus, stem):
    processed_texts = []
    counter = 0
    for i, row in corpus.iterrows():
        clear_output(wait=True)
        print("[%.2f%%] Processing text %d of %d." % (100*(counter+1)/len(corpus), counter+1, len(corpus)))
        counter += 1
        
        text = " ".join( [   
                (w if not stem else STEMMER.stem(w)) 
                    for w in nltk.tokenize.word_tokenize(row["text"]) if w not in STOPWORDS and w not in string.punctuation
        ] )
        processed_texts.append({"text": text, "label": row["label"]})
    return pd.DataFrame(processed_texts)

norm_data = normalize_texts(data, stem=True)
norm_data

[99.99%] Processing text 7200 of 7200.


,text,label
0,após denúnc vej irmã aéci publ víde chor `` vam prov ment '' após ser denunci matér revist vej nest fim seman sen aéci nev mostr indign acus receb de acord revist benedict juni execu odebrecht afi...,1
1,quatr pesso fic fer acid carr caminh br-376 motor automó sofr fer grav resgat helicópter samu nest terça-f 23 em sarand carr peg fog mandaguaçu ônibu escol bat contr mot quatr pesso fic fer acid c...,0
2,mor aceit denúnc contr lul outr 12 cas envolv síti atiba segund força-taref lav jat empreit odebrecht oa compr pag melh síti form propin ex-presid o juiz sérgi mor aceit nest terça-f 1º denúnc con...,0
3,reun urgent for sp dilm cheg capit paul encontr lul a presid futur ex-presid dilm rousseff ness tard dest são paul encontr lul dilm vai hosped hotel renaissanc ond far encontr `` secret '' ... tão...,1
4,no prim dia 2017 antig próx espírit sant orelh michel miguel eli tem lul deu-lh conselh chef ent parec promis com alç massacr result 56 mort complex penitenci anísi jobim manau am dev faz part pre...,0
...,...,...
7195,"um corr seguranç arm divid gabinet juiz sérgi mor sal audi segund and edifíci justiç feder curitib uma câm acopl comput registr prim depo açã penal luiz ináci lul silv réu acus receb r 3,7 milhã o...",0
7196,os rela propost pod alter sistem eleitor brasil próx ele admit óbvi ont dur “ fórum estad – reform polít debat ” realiz parc centr lideranç públic discuss send feit congress pan fund inter parlame...,0
7197,evandr mesquit protest cham sen verm `` quer cal sérgi mor '' ao ver problem polít econôm brasil mesquit mostr artist poli mantém anten lig real o at can declar ser contr trup sen petist formaliz ...,1
7198,investig mostr real caus alt preç pedági paran diz procur lav jat 48ª fas deflagr nest quinta-f 22 seil pesso pres o procur minist públic feder mpf carl fern sant lim afirm manhã dest quinta-f 22 ...,0


In [179]:
# k-fold cross-validation
k = 10
folds = np.split(norm_data.sample(frac=1), k)

accuracies = []
for i in range(len(folds)):
    # separating data
    test_data, test_labels = folds[i]["text"].values, folds[i]["label"].values
    training_data = np.concatenate( [folds[j]["text"].values for j in range(len(folds)) if j != i] )
    training_labels = np.concatenate( [folds[j]["label"].values for j in range(len(folds)) if j != i] )

    # extracting features
    vectorizer = CountVectorizer(max_features=1000)
    training_data = vectorizer.fit_transform(training_data).toarray()  # fit the vectorizer to the training corpus
    test_data = vectorizer.transform(test_data).toarray()  # words of the test corpus that don't appear in the training corpus will be ignored!

    # preparing model
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(32, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(1e-3)),
        tf.keras.layers.Dense(1, activation="sigmoid")
    ])
    model.compile(loss="binary_crossentropy",
                  optimizer=tf.keras.optimizers.Adam(1e-3),
                  metrics=["accuracy"])
    
    # training
    model.fit(training_data, training_labels, epochs=50, 
              validation_data=(test_data, test_labels), 
              callbacks=[ClearCallback(i + 1, k)])
    
    # evaluating
    loss, acc = model.evaluate(test_data, test_labels)
    accuracies.append(acc)

clear_output(wait=True)
print("\n Cross-validation finished! Results:")
print(" . Mean accuracy: %.2f%%" % (100*np.mean(accuracies)))
print(" . Accuracies std: %.2f%%" % (100*np.std(accuracies)))

Running 10-folds cross-validation. Current fold: 10.

23/23 [==============================] - 0s 1ms/step - loss: 0.3830 - accuracy: 0.8958

 Cross-validation finished! Results:
 . Mean accuracy: 88.65
 . Accuracies std: 1.61


# WORD EMBEDDINGS #